In [1]:
pip install streamlit yfinance pandas numpy plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.9 MB/s eta 0:00:00


In [12]:
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px

st.title("Investment Recommendation App: Plan Your Financial Future")

# User inputs


salary = st.number_input("Enter your monthly salary:", min_value=0, key="salary")
savings_percent = st.slider("What percentage of your salary would you like to save?", 0, 100, 20, key="savings_percent")
st.write("Saving consistently is key to building wealth. This slider helps you determine how much you can allocate towards savings.")
savings_amount = salary * (savings_percent / 100)
investment_amount = salary - savings_amount


st.write(f"Your estimated monthly savings amount: ${savings_amount:.2f}")
st.write(f"Amount available for investment after savings: ${investment_amount:.2f}")


# List of stock tickers
tickers = ["ADANIENT.NS", "ADANIPORTS.NS", "APOLLOHOSP.NS", "ASIANPAINT.NS", "AXISBANK.NS",
           "BAJAJ-AUTO.NS", "BAJFINANCE.NS", "BAJAJFINSV.NS", "BPCL.NS", "BHARTIARTL.NS",
           "BRITANNIA.NS", "CIPLA.NS", "COALINDIA.NS", "DIVISLAB.NS", "DRREDDY.NS", "EICHERMOT.NS",
           "GRASIM.NS", "HCLTECH.NS", "HDFCBANK.NS", "HDFCLIFE.NS", "HEROMOTOCO.NS", "HINDALCO.NS",
           "HINDUNILVR.NS", "ICICIBANK.NS", "ITC.NS", "INDUSINDBK.NS", "INFY.NS", "JSWSTEEL.NS",
           "KOTAKBANK.NS", "LTIM.NS", "LT.NS", "M&M.NS", "MARUTI.NS", "NTPC.NS", "NESTLEIND.NS",
           "ONGC.NS", "POWERGRID.NS", "RELIANCE.NS", "SBILIFE.NS", "SHRIRAMFIN.NS", "SBIN.NS",
           "SUNPHARMA.NS", "TCS.NS", "TATACONSUM.NS", "TATAMOTORS.NS", "TATASTEEL.NS", "TECHM.NS",
           "TITAN.NS", "ULTRACEMCO.NS", "WIPRO.NS"]



In [19]:
import datetime

@st.cache
def get_stock_data(ticker, *args, **kwargs):
    today = datetime.date.today()
    end_date = kwargs.get("end_date", today - datetime.timedelta(days=2))  # Get data up to yesterday

    try:
        data = yf.download(ticker, *args, end=end_date.strftime('%Y-%m-%d'))
        return data
    except Exception as e:
        st.error(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

2024-07-26 21:30:43.889 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems or unexpected behavior in certain edge cases.

2024-07-26 21:30:43.892 No runtime found, using MemoryCacheStorageManager


In [20]:
# Load data for each ticker
stock_data = {ticker: get_stock_data(ticker) for ticker in tickers}

# Calculate daily returns and risk
def calculate_returns(data):
    if data.empty:
        return np.nan, np.nan
    data['Return'] = data['Adj Close'].pct_change()
    avg_return = data['Return'].mean()
    risk = data['Return'].std()
    return avg_return, risk

investment_analysis = {}
for ticker in tickers:
    avg_return, risk = calculate_returns(stock_data.get(ticker, pd.DataFrame()))
    investment_analysis[ticker] = {'Average Return': avg_return, 'Risk': risk}

analysis_df = pd.DataFrame(investment_analysis).T
st.write("Investment Analysis:")
st.write(analysis_df)

2024-07-26 21:30:55.172 No runtime found, using MemoryCacheStorageManager
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**

In [15]:
# Recommend investment based on risk-adjusted return
def recommend_investment(analysis_df):
    analysis_df = analysis_df.dropna()  # Remove stocks with missing data
    if analysis_df.empty:
        st.warning("No data available for analysis.")
        return analysis_df
    analysis_df['Risk-Adjusted Return'] = analysis_df['Average Return'] / analysis_df['Risk']
    return analysis_df.sort_values(by='Risk-Adjusted Return', ascending=False)

recommendations = recommend_investment(analysis_df)
st.write("Investment Recommendations:")
st.write(recommendations)

if not recommendations.empty:
    top_recommendation = recommendations.index[0]
    st.write(f"Top investment recommendation: {top_recommendation}")

In [22]:
# Interactive Stock Price Chart
st.subheader("Explore Historical Stock Prices")
selected_stock = st.selectbox("Select a stock to view its price trend:", list(stock_data.keys()))
if selected_stock in stock_data:
    fig = px.line(stock_data[selected_stock], x=stock_data[selected_stock].index, y="Adj Close", title=f"{selected_stock} Stock Price")
    st.plotly_chart(fig)
else:
    st.write(f"Data not available for {selected_stock}.")

In [23]:
# Plot historical stock prices
st.subheader("Stock Price Trends")
selected_stock = st.selectbox("Select a stock to view its price trend:", tickers)

if selected_stock in stock_data:
    fig = px.line(stock_data[selected_stock], x=stock_data[selected_stock].index, y="Adj Close", title=f"{selected_stock} Stock Price")
    st.plotly_chart(fig)
else:
    st.write(f"Data not available for {selected_stock}.")

In [25]:
!pip freeze > requirements.txt
# Download the requirements.txt file
from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>